## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [3]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline

# Весь код, который нужно разобрать, будем дебажить

In [4]:
x=np.array([[3,2],[4,4],[2,4]])

In [5]:
y=np.array([[1], [2], [2]])

In [6]:
sorted_idx = x.argsort()
sorted_idx

array([[1, 0],
       [0, 1],
       [0, 1]], dtype=int64)

сортируем внутри строк

In [7]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([[[4, 4],
         [3, 2]],
 
        [[3, 2],
         [4, 4]],
 
        [[3, 2],
         [4, 4]]]), array([[[2],
         [1]],
 
        [[1],
         [2]],
 
        [[1],
         [2]]]))

пока непонятно что, скорее всего передаем ОДИН признак. Попробуем еще раз

In [8]:
x=np.array([7, 6, 5,4,3,2,1,0])
y=np.array([1, 1, 1,1,0,0,2,2])

In [9]:
sorted_idx = x.argsort()
sorted_idx

array([7, 6, 5, 4, 3, 2, 1, 0], dtype=int64)

In [10]:
sorted_x, sorted_y = x[sorted_idx], y[sorted_idx]
sorted_x, sorted_y 

(array([0, 1, 2, 3, 4, 5, 6, 7]), array([2, 2, 0, 0, 1, 1, 1, 1]))

Отсортировали значения по возрастанию признака

In [11]:
class_number = np.unique(y).shape[0]
class_number

3

In [12]:
np.unique(y)

array([0, 1, 2])

Получили количество уникальный классов

In [13]:
splitted_sorted_y = sorted_y
splitted_sorted_y

array([2, 2, 0, 0, 1, 1, 1, 1])

удалил лишнее обрезание, так как это должно выполняться в другом методе

In [14]:
splitted_sorted_y = sorted_y
r_border_ids = np.where(splitted_sorted_y[:-1] != splitted_sorted_y[1:])[0] + (1)

r_border_ids

array([2, 4], dtype=int64)

получили список, где будем делить (правые границы изменения классов)

Если класс по размеру не разделим, то возвращаем float('+inf'), None

In [15]:
one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
one_hot_code

array([[0., 0., 0.],
       [0., 0., 0.]])

Кодируем классы - количество строк - количество возможных делений, количество колоннок - количество классов

In [16]:
eq_el_count = r_border_ids - np.append([0], r_border_ids[:-1])
eq_el_count

array([2, 2], dtype=int64)

количество одинаковых элементов (после отсечения)

In [17]:
np.arange(r_border_ids.shape[0])

array([0, 1])

получаем индексы для всех строк

In [18]:
sorted_y[r_border_ids - 1]

array([2, 0])

получаем значения левых границ

классы должны быть закодированы, начиная с 0

In [19]:
one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
one_hot_code

array([[0., 0., 1.],
       [1., 0., 0.]])

In [20]:
eq_el_count.reshape(-1, 1)

array([[2],
       [2]], dtype=int64)

In [21]:
class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
class_increments        

array([[0., 0., 2.],
       [2., 0., 0.]])

In [22]:
np.bincount(sorted_y[:], minlength=class_number)

array([2, 4, 2], dtype=int64)

In [23]:
class_increments[0] = class_increments[0] 
class_increments

array([[0., 0., 2.],
       [2., 0., 0.]])

Получили количество классов КРОМЕ находящиегося справа на ВСЕЙ выборке

In [24]:
l_class_count = np.cumsum(class_increments, axis=0)        
l_class_count

array([[0., 0., 2.],
       [2., 0., 2.]])

наконец-то получили количество классов слева при разделении

In [25]:
r_class_count = np.bincount(y) - l_class_count
r_class_count

array([[2., 4., 0.],
       [0., 4., 0.]])

получили количество классов справа

In [26]:
l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
l_sizes

array([[2],
       [4]], dtype=int64)

сумма классов слева

In [27]:
r_sizes = sorted_y.shape[0] - l_sizes
r_sizes

array([[6],
       [4]], dtype=int64)

сумма классов справа

С кодом разобрались. Теперь тестируем критерии информативности.
Так как берется левая и правая часть и берется минимум, то просто берем сумму левой и правой части по информативности

In [28]:
    def gini(l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        
        sum_l = np.sum(np.power(l_c, 2), axis=1)
        c_l = np.power(l_s.reshape([1, l_s.shape[0]*l_s.shape[1]])[0], 2)
        
        sum_r = np.sum(np.power(r_c, 2), axis=1)
        c_r = np.power(r_s.reshape([1, r_s.shape[0]*r_s.shape[1]])[0], 2)
        return 2 - sum_l/c_l - sum_r/c_r

In [29]:
gini(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.44444444, 0.5       ])

In [30]:
l_class_count, l_sizes, r_class_count, r_sizes

(array([[0., 0., 2.],
        [2., 0., 2.]]), array([[2],
        [4]], dtype=int64), array([[2., 4., 0.],
        [0., 4., 0.]]), array([[6],
        [4]], dtype=int64))

Проверим

In [31]:
1-4/4+1-4/36-16/36, 1-8/16+1-16/16

(0.4444444444444444, 0.5)

Совпало. Идем дальше

In [32]:
l_class_count/l_sizes

array([[0. , 0. , 1. ],
       [0.5, 0. , 0.5]])

In [33]:
def entropy(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    p_ll = np.log2(p_l, where = p_l!=0)
    p_rl = np.log2(p_r,  where = p_r!=0)
    return - np.sum(p_l*p_ll, axis=1) - np.sum(p_r*p_rl, axis=1) 

In [34]:
entropy(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.91829583, 1.        ])

In [35]:
def misclass(l_c, l_s, r_c, r_s):
    p_l = l_c/l_s
    p_r = r_c/r_s
    m_l = np.max(p_l, axis=1)
    m_r = np.max(p_r, axis=1)
    return 2 - m_l - m_r

In [36]:
misclass(l_class_count, l_sizes, r_class_count, r_sizes)

array([0.33333333, 0.5       ])

In [37]:
import math

Теперь определения индексов feature

In [38]:
def get_feature_ids_sqrt(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(n_feature**0.5)]
        
def get_feature_ids_log2(n_feature):
    feature_ids = np.arange(n_feature)
    np.random.shuffle(feature_ids)
    return feature_ids[:int(math.log2(n_feature))]

def get_feature_ids_N(n_feature):
    return np.arange(n_feature)

In [39]:
get_feature_ids_sqrt(4)

array([3, 2])

In [40]:
get_feature_ids_log2(8)

array([7, 4, 3])

In [41]:
get_feature_ids_N(3)

array([0, 1, 2])

все кажется нормально

Дальше идут куски тестирования итеративного алгоритма

In [42]:
ar = np.array([1,3,4,3,3,0])

In [43]:
np.max(np.unique(ar, return_counts=True)[1])/ar.shape[0]

0.5

In [44]:
clas, count = np.unique(ar, return_counts=True)

In [45]:
clas, count

(array([0, 1, 3, 4]), array([1, 1, 3, 1], dtype=int64))

In [63]:
np.argmax(count)

2

In [64]:
xx = np.array([[1,2],[2,3]])

In [66]:
xx[:,0]

array([1, 2])

In [164]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, 
                 sufficient_share=1.0, criterion='gini', max_features=None):
        #комментарии для себя:
        #ожидается в dict:
        #ключ - id, как при хранении дерева в массиве, так:
        # 0 - слева 1, справа 2
        # 1 - слева 2*1+1=3, справа 2*1+2=4
        # 2 - слева 2*2+1=5, справа 2*2+2 = 6
        # В самом объекте ожидается массив из 3 элементов, 
        # первый - код LEAF_TYPE, промежуточные NON_LEAF_TYPE, конечные - LEAF_TYPE
        # для NON_LEAF_TYPE - 2 и 3 - feature_id, threshold
        # для LEAF_TYPE - класс и вероятность
        self.tree = dict()
        
        
        # минимальное количество элементов в листе для разделения
        self.min_samples_split = min_samples_split
        #максимальная глубина
        self.max_depth = max_depth
        
        if self.max_depth==None:
            self.max_depth = 20
        #доля преобладающего класса
        self.sufficient_share = sufficient_share
        self.num_class = -1
        #функция критерии
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print ('invalid criterion name')
            raise

        #количество используемых призаков
        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print ('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        #l_class_count, l_sizes, r_class_count, r_sizes
        #См. выше в отладке
        return gini(l_c, l_s, r_c, r_s)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return entropy(l_c, l_s, r_c, r_s)

    def __misclass(self, l_c, l_s, r_c, r_s):
        #См. выше в отладке
        return misclass(l_c, l_s, r_c, r_s)

    def __get_feature_ids_sqrt(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_sqrt(n_feature)
    
    def __get_feature_ids_log2(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_log2(n_feature)

    def __get_feature_ids_N(self, n_feature):
        #См. выше в отладке
        return get_feature_ids_N(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        # см. выше
        sorted_x, sorted_y = self.__sort_samples(x, y)
        class_number = np.unique(y).shape[0]
        
        #  см. выше
        splitted_sorted_y = sorted_y
        r_border_ids = np.where((splitted_sorted_y[:-1] != splitted_sorted_y[1:]) & (sorted_x[:-1] != sorted_x[1:]) )[0] + (1)
        #здесь добавим код, который говорит, что и x должны быть разные
        #print('sorted_y')
        #print(sorted_y)

        #print('r_border_ids')
        #print(r_border_ids)
        if len(r_border_ids) == 0:
            return None, None
        
        #  см. выше
        eq_el_count = r_border_ids - np.append([0], r_border_ids[:-1])
        one_hot_code = np.zeros((r_border_ids.shape[0], class_number))
        one_hot_code[np.arange(r_border_ids.shape[0]), sorted_y[r_border_ids - 1]] = 1
        class_increments = one_hot_code * eq_el_count.reshape(-1, 1)
        class_increments[0] = class_increments[0] 
        
        #  см. выше
        l_class_count = np.cumsum(class_increments, axis=0)        
        r_class_count = np.bincount(y) - l_class_count
        l_sizes = r_border_ids.reshape(l_class_count.shape[0], 1)
        r_sizes = sorted_y.shape[0] - l_sizes

        
        #как-то считаем критерий информативности, при этом возвращаем массив [1,2,34]
        #print('\n')
        #print(l_class_count, l_sizes, r_class_count, r_sizes)
        gs = self.G_function(l_class_count, l_sizes, r_class_count, r_sizes)
        #находим минимальное значение
        idx = np.argmin(gs)
    
        # значение границы
        left_el_id = l_sizes[idx][0]
        #print('left_el_id')
        #print(left_el_id)
        #print('sorted_x[left_el_id-1]', 'sorted_x[left_el_id]')
        #print(sorted_x[left_el_id-1], sorted_x[left_el_id])
        #возвращаем минимальное значение информативности и значение границы
        return gs[idx], (sorted_x[left_el_id-1] + sorted_x[left_el_id]) / 2.0

    def __fit_node(self, x_m, y_m, node_id_m, depth_m):
        # Ваш код
        # Необходимо использовать следующее:
        # self.LEAF_TYPE
        # self.NON_LEAF_TYPE

        # self.tree
        # self.max_depth
        # self.sufficient_share
        # self.min_samples_split

        # self.get_feature_ids
        # self.__find_threshold
        # self.__div_samples
        # self.__fit_node
        
        lst = [[x_m, y_m, node_id_m, depth_m]]
        
        while len(lst)>0:
            x, y, node_id, depth = lst.pop()
            #придется делать через рекурсию, так как такой интерфейс, хотя для python не очень хорошо
            #сначала определим что мы НЕ в LEAF:
            type_leaf = self.NON_LEAF_TYPE
            if self.max_depth is not None and depth >= self.max_depth:
                type_leaf = self.LEAF_TYPE

            share = np.max(np.unique(y, return_counts=True)[1])/y.shape[0]
            if share >= self.sufficient_share:
                type_leaf = self.LEAF_TYPE
            if y.shape[0] <= self.min_samples_split:
                 type_leaf = self.LEAF_TYPE

            if type_leaf == self.LEAF_TYPE:
                self.__process_leaf_node(y, node_id)
                continue

            #дальше определяем по каким признакам будем считать
            features = self.get_feature_ids(x.shape[1])
            find_feature = 0
            find_value = None
            find_threshold = None
            find_gs = None
            for feature in features:
                value, threshold = self.__find_threshold( x[:, feature], y)
                if value == None:
                    continue
                if find_value==None or find_value > value:
                    find_feature = feature
                    find_value = value
                    find_threshold = threshold
            #print('x[:, feature]')
            #print(x[:, find_feature])
            #print('y')
            #print(y)
            #print('value', 'find_threshold')
            #print(value, find_threshold)

            if find_value == None:
                #случай ошибки, если не смогли найти разделения
                self.__process_leaf_node(y, node_id)
                return


            x_l, x_r, y_l, y_r =self.__div_samples(x, y, find_feature, find_threshold)

            self.__process_non_leaf_node(node_id, find_feature, find_threshold)

            #print('len(y_l)', 'len(y_r)')
            #print(len(y_l), len(y_r))
            #print(np.unique(y_l), np.unique(y_r))
            ##учим левую половину
            lst.append([x_l, y_l, 2*node_id+1, depth+1])
            lst.append([x_r, y_r, 2*node_id+2, depth+1])


    def __process_leaf_node(self, y, node_id):
        clas, count = np.unique(y, return_counts=True)
        max_class = clas[np.argmax(count)]
        probability = np.max(count) / y.shape[0]
        #print(node_id)
        self.tree[node_id] = [self.LEAF_TYPE, max_class , probability]
    
    def __process_non_leaf_node(self, node_id, feature, threshold):
        #print(node_id)
        self.tree[node_id] = [self.NON_LEAF_TYPE, feature , threshold]
        
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        self.__fit_node(x, y, 0, 0) 

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [150]:
df = pd.read_csv('cs-training.csv', sep=',').dropna()
df.shape

(120269, 11)

In [151]:
x = df.as_matrix(columns=df.columns[1:])
y = df.as_matrix(columns=df.columns[:1])
y = y.reshape(y.shape[0])

C:\app\anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
C:\app\anaconda\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [152]:
np.unique(y, return_counts=True)[1]

array([111912,   8357], dtype=int64)

In [132]:
111912/(111912+8357)

0.930514097564626

In [171]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, max_depth=8)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы

In [176]:
t1 = time()
my_clf.fit(x, y)
t2 = time()
print(t2 - t1)
print(my_clf.tree)


0.8130269050598145
{0: [0, 5, 56.5], 2: [0, 1, 100.0], 6: [0, 4, 243745.0], 14: [0, 7, 30.5], 30: [0, 4, 234800.0], 62: [0, 4, 226637.0], 126: [0, 7, 27.5], 254: [0, 7, 25.5], 510: [1, 0, 0.9305337225840258], 509: [1, 0, 1.0], 253: [1, 1, 1.0], 125: [1, 1, 1.0], 61: [1, 0, 1.0], 29: [1, 0, 1.0], 13: [0, 5, 16.5], 28: [0, 3, 0.023822465], 58: [0, 3, 0.017881928499999998], 118: [1, 0, 1.0], 117: [1, 1, 1.0], 57: [1, 0, 1.0], 27: [1, 0, 1.0], 5: [0, 7, 1.0], 12: [0, 9, 0.5], 26: [0, 1, 101.5], 54: [1, 0, 0.5], 53: [1, 0, 1.0], 25: [1, 0, 1.0], 11: [1, 0, 1.0], 1: [0, 5, 57.5], 4: [1, 0, 0.5], 3: [1, 0, 1.0], 508: [1, 0, 0.8333333333333334], 507: [1, 0, 1.0], 124: [1, 0, 0.5], 123: [1, 0, 1.0], 252: [1, 0, 0.5], 506: [1, 0, 1.0], 505: [1, 1, 1.0], 251: [1, 0, 1.0], 60: [1, 0, 0.5], 59: [1, 0, 1.0]}


In [177]:

t1 = time()
clf.fit(x, y)
t2 = time()
print(t2 - t1)

0.8730006217956543


## Проверка качества работы

In [168]:
gkf = KFold(n_splits=5, shuffle=True)

In [178]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    my_clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test))

0.9291593913694188
0.934979629167706
0.9300740001662925
0.930905462708905
0.9272440028270902


In [ ]:
for train, test in gkf.split(x, y):
    X_train, y_train = x[train], y[train]
    X_test, y_test = x[test], y[test]
    clf.fit(X_train, y_train)
    print(accuracy_score(y_pred=clf.predict(X_test), y_true=y_test))

0.8895817743410659


# Применить для задачи Titanic 